In [22]:
# Importamos lo importante del proyecto
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as tv_models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import ayuda

In [26]:
# Configuración del dispositivo (CPU o GPU)
dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilizando: {dispositivo}")

Utilizando: cpu


In [6]:
# Manejo de imágenes truncadas
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
# Cargamos el directorio del dataset
dataset = r'C:\Users\Marc\Desktop\Proyecto Dectector AI\archive'

In [8]:
# Función para crear el dataset
def crear_dataset(dataset):
    ruta_entrenamiento = dataset + "/train"
    ruta_validacion = dataset + "/test"
    
    entrenamiento_dataset = ImageFolder(root=ruta_entrenamiento)
    validacion_dataset = ImageFolder(root=ruta_validacion)
    
    return entrenamiento_dataset, validacion_dataset

In [9]:
# Definimos la media y la desviación estándar de ImageNet
media_imagenet= torch.tensor([0.485, 0.456, 0.406])
desviacion_imagenet= torch.tensor([0.229, 0.224, 0.225])

In [10]:
# Definimos las transformaciones para las imágenes (entrenamiento y validación)
def transformaciones(media=media_imagenet, desviacion=desviacion_imagenet):
    
    entrenamiento_transform = transforms.Compose([
        transforms.Resize((224, 224)), # Redimensionar a 224x224
        transforms.RandomHorizontalFlip(p=0.5), # Flip horizontal aleatorio
        transforms.ColorJitter(brightness=0.2, contrast=0.2), # Ajuste de brillo y contraste
        transforms.ToTensor(), # Convertir a tensor
        transforms.Normalize(media,desviacion) # Normalizar
    ])
    
    validacion_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(media,desviacion)])

    return entrenamiento_transform, validacion_transform    

In [11]:
# Creamos los DataLoaders para entrenamiento y validación
def crear_dataloaders(set_entrenamiento, set_validacion, batch_size):
    
    entrenamiento_transformado, validacion_transformado = transformaciones()

    set_entrenamiento.transform = entrenamiento_transformado
    set_validacion.transform = validacion_transformado
    
    # Shuffle en entrenamiento, no en validación
    entrenamiento_cargado = DataLoader(set_entrenamiento, batch_size=batch_size, shuffle=True)
    validacion_cargado = DataLoader(set_validacion, batch_size=batch_size, shuffle=False)
    
    return entrenamiento_cargado, validacion_cargado, set_entrenamiento, set_validacion

In [12]:
# Hacemos fine-tuning del modelo preentrenado ConvNeXt-Tiny
def cargar_MobileNetV3_Large_desde_pesos(ruta_pesos, device="cpu"):
    modelo = tv_models.mobilenet_v3_large(weights=None)  # sin pesos
    state_dict = torch.load(ruta_pesos, map_location=torch.device(device))
    modelo.load_state_dict(state_dict)
    return modelo

In [13]:
pesos_locales = r'C:\Users\Marc\Desktop\Proyecto Dectector AI\mobilenet_v3_large-5c1a4163.pth'
test_modelo = cargar_MobileNetV3_Large_desde_pesos(pesos_locales)

In [14]:
# Actualizamos la capa final para clasificación que nos interesa (2 clases: Imagen real e Imagen IA)
def actualizar_capa_final(modelo, num_clases):
    for caract_parametros in modelo.features.parameters():
        caract_parametros.requires_grad = False  # Congelamos las capas de características
    
    ultima_capa = modelo.classifier[-1] # Obtenemos la última capa
    num_caract = ultima_capa.in_features # Número de características de entrada
    nueva_clasificacion = nn.Linear(num_caract, num_clases) # Nueva capa lineal
    modelo.classifier[-1] = nueva_clasificacion # Reemplazamos la última capa
    
    return modelo

In [15]:
# Modificamos la ultima capa del modelo cargado
test_modelo = actualizar_capa_final(test_modelo, num_clases=2)

In [16]:
# Finalmente, entrenamos y evaluamos el modelo
entrenamiento_dataset, validacion_dataset = crear_dataset(dataset)
entreanmiento_cargado, validacion_cargado, _, _ = crear_dataloaders(entrenamiento_dataset, validacion_dataset, batch_size=32)

In [17]:
# Cargamos los pesos de MobileNetV3-Large y modificamos su última capa
pesos_locales = r'C:\Users\Marc\Desktop\Proyecto Dectector AI\mobilenet_v3_large-5c1a4163.pth'
MobileNetV3_Large_modelo = cargar_MobileNetV3_Large_desde_pesos(pesos_locales)
MobileNetV3_Large_modelo = actualizar_capa_final(MobileNetV3_Large_modelo, num_clases=2)

In [18]:
# Definimos la función de pérdida y el optimizador
funcion_perdida = nn.CrossEntropyLoss()
optimizador = optim.Adam(filter(lambda p: p.requires_grad, MobileNetV3_Large_modelo.parameters()), lr=0.001)

In [27]:
# Entrenamos el modelo
num_epocas = 3
modelo_entrenado = ayuda.entrenamiento_loop(
    MobileNetV3_Large_modelo,
    entreanmiento_cargado,
    validacion_cargado,
    funcion_perdida,
    optimizador,
    dispositivo,
    num_epocas=2
)

Epoca 1/2 Entrenando:   0%|          | 0/1500 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [28]:
# Mostramos algunas predicciones del modelo entrenado
ayuda.mostrar_predicciones(
    modelo_entrenado,
    validacion_cargado,
    dispositivo,
    n=6
)

NameError: name 'modelo_entrenado' is not defined

In [4]:
# Probamos el modelo con una imagen local
ayuda.predecir_imagen_local(
    modelo_entrenado,
    r"C:\Users\Marc\Desktop\Proyecto Dectector AI\imagenes_prueba\9.jpg",
    validacion_cargado.dataset.transform,
    dispositivo,
    validacion_cargado.dataset.classes
)

NameError: name 'modelo_entrenado' is not defined